Central unit of data is a tensor, which is a set of primitive data with any number of dimensions, and these dimensions are called a tensor's rank.

In [1]:
import tensorflow as tf

There are two crucial sections to using tensorflow. 
1. Building the computational graph
2. Running the computational graph

A computational graph is a series of operations arranged into a graph of nodes. Each node takes a tensor as input and outputs a tensor, except there is one type that is constant and outputs itself.

Here is how to create two floating point tensors: (note that they are declared with tf.constant, which means that they are constants) 

In [3]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly, so we do not need to specify if we want default 32-float
print(node1, node2)

Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32)


Simply printing the nodes does not produce 3.0 and 4.0. To get this, we must evaluate the nodes by running the computational graph within a session. We have already created a computational graph with our two nodes, so we create a session and the run the graph through the session, which encapsulates the control and runflow of Tensorflow.

In [4]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


Now we have the constants that we assigned the nodes when we initialized them. We can also do operations with our nodes (an operation is also a node). For example, we can add the two nodes together to make a new graph.


In [5]:
node3 = tf.add(node1, node2)

Notice that we get similar outputs to what we previously for when we simply try to print the node...

In [6]:
print("node3: ", node3)

node3:  Tensor("Add:0", shape=(), dtype=float32)


...versus running the node through a session and printing the results.

In [7]:
print("sess.run(node3): ", sess.run(node3))

sess.run(node3):  7.0


We're going to make a graph of our computation using TensorBoard. Placeholders allow the graph to accept external inputs, which is like a promise for a value later.

In [8]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b #this step is a shortcut to tf.add(a, b)

In [9]:
print(sess.run(adder_node, {a: 3, b: 4.5}))

7.5


In the next line, we can see the placeholder allowing us to evaluate the adder_node at multiple values by using only one line.

In [11]:
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

[ 3.  7.]


TensorBoard is going to be covered in another notebook, as it turns out it is more complex and requires more attention than the brevity of this current notebook.

We can make the graph more complex by adding another operation.

In [12]:
add_and_triple = adder_node * 3
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


We can still do this with multiple values in one line just as before.

In [13]:
print(sess.run(add_and_triple, {a: [1, 3], b: [2, 4]}))

[  9.  21.]


Before we used constants to make our graph. Now were going to use variables, which will be able to take arbitrary inputs. This will allow our model to become trainable, as we need to be able to modify the graph to get new outputs with the same input.

In [14]:
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

Constants are initialized when you call tf.constant, and their value can never change. Variables are not initialized when you call tf.Variable and must be initialized in a special way or they will stay unitialized even if we assign values to them, as above.

In [15]:
init = tf.global_variables_initializer()

sess.run(init)

Because x is our placeholder, we can evaluate the linear_model for several values of x like we did before, using a list:

In [17]:
print(sess.run(linear_model, {x:[1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


Doing this, we have created a model but there is no way to tell how good it is, so to evaluate it, we will make another placeholder value to give us the desired values and evaluate the performance. We will also write a loss function.

A loss function provides a measurement between where the current model is and where the provided data is. We'll use a standard loss model for linear regression, which sums the squares of the delats between where the current model is and the provided data.

linear_model - y = a vector of elements where each elements is the corresponding example's error delta

tf.square will square that error.

tf.reduce_sum will sum all squared error to create a single scalar that abstracts the error of all examples.

In [18]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:[1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


This can be manually improved by reassigning the values of W and b to perfect values of -1 and 1. If a variable is initialized to a value, it can be reassigned using tf.assign.

In [19]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1, 2, 3, 4], y:[0, -1, -2, -3]}))

0.0


### Examples

The example below uses both a constant and a variable in its graph. It also uses a simple way of initializing the variables. The result is expected, as a list of integers that have been modified by the variable y.

In [3]:
import tensorflow as tf

x = tf.constant([35, 40, 45], name = 'x')
y = tf.Variable(x + 5, name  = 'y')
model = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(model)
    print(session.run(y))

[40 45 50]


Below is an exercise that has been completed. The goal is to generate a numpy array of 10000 random integers (called x) and create a Variable storing the equation y = 5x^2 - 3x + 15. Numpy should be used for larger lists/arrays, as it is more memory efficient and faster to compute on than lists. It also has more functions available, such as mean, that are not normally available to lists.

In [5]:
import numpy as np

data = np.random.randint(1000, size = 10000)

x = tf.constant(data, name = 'x')
y = tf.Variable(5*x**2 - 3*x + 15)
model = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(model)
    print(session.run(y))

[3635501 1213781  918933 ...,  145707  550139   48723]


The following example was meant to be run and then analyzed.

In [6]:
x = tf.Variable(0, name='x')

model = tf.global_variables_initializer()

with tf.Session() as session:
    for i in range(5):
        session.run(model)
        x = x + 1
        print(session.run(x))

1
2
3
4
5


For the example above, we were able to create these numbers by starting with a variable whose value was zero, creating a loop that terminates after 5 iterations, and changing the value of the variable by adding 1 with each loop.

The next exercise is as follows: Using the code from (2) and (3) above (or the previous two programs), create a program that computers the “rolling” average of the following line of code: np.random.randint(1000). In other words, keep looping, and in each loop, call np.random.randint(1000) once in that loop, and store the current average in a Variable that keeps updating each loop.

In [22]:
x = tf.Variable(0, name='x')
total = 0
average = 0

model = tf.global_variables_initializer()

with tf.Session() as session:
    for i in range(5):
        r = np.random.randint(1000)
        total = total + r
        average = total / (i + 1)
        tx = tf.assign(x, int(average))
        session.run(model)
        print(session.run(tx))

452
516
581
519
421


For the code above: tf.assign(ref, value), be careful that value is the right type. Because of the average operation (average = total / (i + 1)), the division involved caused a type error, that assign was expecting value to be an integer but it was not. Type conversion (int(average)) had to be applied so that the method would work.